# Manager Agent E2E Test + RAGAS Evaluation

In [1]:
# 1. env & 라이브러리
from pathlib import Path
from dotenv import load_dotenv
import sys

ROOT = Path.cwd().parent  # 노트북은 tests/ 하위에 있다고 가정
sys.path.append(str(ROOT))

load_dotenv(ROOT / ".env")
print(f"ROOT - {ROOT}")
print("환경 변수 로드 완료")

from services.orchestrator import router_node
from ragas import EvaluationDataset, SingleTurnSample, evaluate
from ragas.metrics import answer_relevancy, faithfulness, LLMContextPrecisionWithoutReference
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI, OpenAIEmbeddings



ROOT - c:\Users\insung\Finance_Agent
환경 변수 로드 완료


2025-08-04 13:00:18,493 | INFO | Loading faiss with AVX2 support.
2025-08-04 13:00:18,538 | INFO | Successfully loaded faiss with AVX2 support.
2025-08-04 13:00:18,551 | INFO | Failed to load GPU Faiss: name 'GpuIndexIVFFlat' is not defined. Will not load constructor refs for GPU indexes. This is only an error if you're trying to use GPU Faiss.


In [2]:
# 2. Graph 호출 → Samples 변환

# 인덱스 준비되어 있다고 전제
SAMPLE_QUESTIONS = [
    # FAQ 타입
    "OTP 비밀번호 오류 해제 방법 알려줘",
    "해외에서 인터넷뱅킹 이용 시 주의할 점은?",
    # 상품 설명
    "우리 SUPER주거래 적금 3년 만기 금리가 얼마야?",
    "정기적금 우대이율 조건 요약해줘",
    # 가입/추천
    "나에게 맞는 적금 상품 추천해줘",
    "청년이라면 어떤 예금이 유리할까?"
]


samples = []
for q in SAMPLE_QUESTIONS:
    res = router_node.invoke(q)
    ctxs = res.get("context", "").split("\n\n")
    samples.append(
        SingleTurnSample(
            user_input         = q,
            response           = res["answer"],
            retrieved_contexts = ctxs,
        )
    )
dataset = EvaluationDataset(samples=samples)


2025-08-04 13:00:25,868 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-04 13:00:26,851 | INFO | HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08-04 13:00:29,001 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-04 13:00:29,725 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-04 13:00:30,533 | INFO | HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08-04 13:00:34,164 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-04 13:00:35,210 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-04 13:00:35,552 | INFO | HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08-04 13:00:36,229 | INFO | HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08

In [3]:
import pprint
pprint.pp(samples)

[SingleTurnSample(user_input='OTP 비밀번호 오류 해제 방법 알려줘', retrieved_contexts=['Q: 보안카드(OTP발생기) 비밀번호 오입력횟수가 초과되었어요. 인터넷뱅킹에서 오류해제 가능한가요?\nA: 보안카드 비밀번호 연속 3회 또는 OTP발생기 비밀번호 연속 10회 오입력 초과 시 보안매체를 이용할 수 없습니다. 우리WON뱅킹에서 비대면 실명확인 절차로 오류해제 가능합니다. (인터넷뱅킹에서는 불가)', 'Q: 이용자비밀번호 3회 오류 상태인데, 인터넷뱅킹에서 오류 해제 할 수 있나요?\nA: 기존 이용자비밀번호를 알고 있다면 인터넷뱅킹에서 오류해제 할 수 있습니다. (보안매체 및 인증서 입력 필요)', '▶ 인터넷뱅킹 경로 안내 : 개인뱅킹 로그인→ (아이디 로그인 영역) 비밀번호 재등록→ 이용자비밀번호 3회 오류해제', '※ 이용자비밀번호 3회오류인 경우, 아이디로그인은 불가하나 인증서 등 다른 방법의 로그인은 가능합니다.(법인 제외)', '※ 인터넷뱅킹에서 오류해제가 불가하다면, 우리WON뱅킹에서 비대면 실명확인 절차로 오류해제 및 비밀번호 재등록을 할 수 있습니다.', 'Q: 이용자비밀번호(로그인비밀번호) 2회 오류 상태인데, 인터넷뱅킹에서 재등록 할 수 있나요?\nA: 인터넷뱅킹에서 이용자비밀번호 재등록이 가능하며, 보안매체와 인증서 입력이 필요합니다.', '', '▶ 이용자비밀번호 재등록 경로 안내 : 개인인터넷뱅킹 → (아이디 로그인 영역) 비밀번호재등록 → 이용자비밀번호 재등록', '', '※ 인터넷뱅킹에서 재등록 불가하다면, 우리WON뱅킹에서 비대면 실명확인 절차로 오류해제 및 비밀번호 재등록을 할 수 있습니다.'], reference_contexts=None, response='OTP 비밀번호 오류 해제 방법은 다음과 같습니다:\n\n1. 우리WON뱅킹에 접속합니다.\n2. 비대면 실명확인 절차를 진행합니다.\n3. 오류가 해제된 후, 필요한 경우 비밀번호를 재등록합니다.\n\n※ 인터넷뱅킹에서는 OT

In [4]:
# 평가 설정
eval_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini", temperature=0))
eval_emb = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

metrics = [
    answer_relevancy,
    faithfulness,
    LLMContextPrecisionWithoutReference(llm=eval_llm),
]

In [5]:
# RAGAS 실행
report = evaluate(
    dataset,
    metrics   = metrics,
    llm       = eval_llm,
    embeddings= eval_emb,
    batch_size= 4,
    show_progress=True,
)
df = report.to_pandas()
df.head()


Evaluating:   0%|          | 0/18 [00:00<?, ?it/s]

Batch 1/5:   0%|          | 0/4 [00:00<?, ?it/s]

2025-08-04 13:04:09,847 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-04 13:04:09,932 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-04 13:04:09,966 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-04 13:04:10,058 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-04 13:04:10,103 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-04 13:04:11,690 | INFO | HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08-04 13:04:12,016 | INFO | HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08-04 13:04:12,178 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-04 13:04:12,184 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200

,user_input,retrieved_contexts,response,answer_relevancy,faithfulness,llm_context_precision_without_reference
0,OTP 비밀번호 오류 해제 방법 알려줘,[Q: 보안카드(OTP발생기) 비밀번호 오입력횟수가 초과되었어요. 인터넷뱅킹에서 오...,OTP 비밀번호 오류 해제 방법은 다음과 같습니다:\n\n1. 우리WON뱅킹에 접속...,0.696997,0.8,0.666667
1,해외에서 인터넷뱅킹 이용 시 주의할 점은?,[Q: 해외에서 인터넷/스마트뱅킹을 이용하려면 출국하기 전에 확인할 사항이 있나요?...,해외에서 인터넷뱅킹을 이용할 때 주의할 점은 다음과 같습니다.\n\n1. 해외IP차...,0.533919,1.0,0.840476
2,우리 SUPER주거래 적금 3년 만기 금리가 얼마야?,[Q: 현재 거주하고 있는 전세주택의 임대차계약으로도 전세대출이 가능한가요?\nA:...,해당 내용은 담당 직원 확인 후 안내하겠습니다(②). 추가적인 정보가 필요하시면 언...,0.000000,0.0,0.000000
3,정기적금 우대이율 조건 요약해줘,[Q: 가족이 방문하여 정기예금 만기해지를 대신 할 수 있나요?\nA: 예금해지는 ...,추가 확인 필요. <context>에는 정기적금 우대이율 조건에 대한 정보가 포함되...,0.000000,0.0,0.190794
4,나에게 맞는 적금 상품 추천해줘,[Q: 어떤 펀드가 전문가 포트폴리오에 추천되나요?\nA: 우리은행에서 자체개발한 ...,고객님의 상황에 따라 적합한 적금 상품을 추천해드리겠습니다.\n\n1. **우리 퍼...,0.000000,0.8,0.601951


## Overall Scores

In [6]:
df_overall = df[[
    "answer_relevancy",
    "faithfulness",
    "llm_context_precision_without_reference"
]].mean()

print(df_overall)

answer_relevancy                           0.283656
faithfulness                               0.550000
llm_context_precision_without_reference    0.395057
dtype: float64


- answer_relevancy ≥ 0.8 이면 FAQ 일치율 양호   낮으면 프롬프트/쿼리 확장/리트리버 k값 조정.

- faithfulness < 0.9 라면 Prompt 수정·retriever k값 조정 필요, ‘컨텍스트 내 근거 인용’ 지시 강화, 답변 길이 제한, 시스템 프롬프트 보강.

- context_precision 은 1.0에 가까울수록 불필요한 문맥이 적음. 임베딩 모델/토크나이저, 인덱스 파라미터(HNSW, k), 쿼리 재작성 확인.

 OOD(미지원 질문)용 거절 정확도 같은 보조 지표도 추가하면 좋을 듯.

## Sample-level (sorted by Faithfulness)

In [7]:
print(df.sort_values("faithfulness")[[
    "user_input",
    "retrieved_contexts",
    'response',
    "answer_relevancy",
    "faithfulness",
    "llm_context_precision_without_reference"
]])

                      user_input  \
3              정기적금 우대이율 조건 요약해줘   
2  우리 SUPER주거래 적금 3년 만기 금리가 얼마야?   
5             청년이라면 어떤 예금이 유리할까?   
0          OTP 비밀번호 오류 해제 방법 알려줘   
4              나에게 맞는 적금 상품 추천해줘   
1        해외에서 인터넷뱅킹 이용 시 주의할 점은?   

                                  retrieved_contexts  \
3  [Q: 가족이 방문하여 정기예금 만기해지를 대신 할 수 있나요?\nA: 예금해지는 ...   
2  [Q: 현재 거주하고 있는 전세주택의 임대차계약으로도 전세대출이 가능한가요?\nA:...   
5  [Q: 미성년자 본인 예금가입 시 부모님과 함께 가야하나요?\nA: 만 14세 이상...   
0  [Q: 보안카드(OTP발생기) 비밀번호 오입력횟수가 초과되었어요. 인터넷뱅킹에서 오...   
4  [Q: 어떤 펀드가 전문가 포트폴리오에 추천되나요?\nA: 우리은행에서 자체개발한 ...   
1  [Q: 해외에서 인터넷/스마트뱅킹을 이용하려면 출국하기 전에 확인할 사항이 있나요?...   

                                            response  answer_relevancy  \
3  추가 확인 필요. <context>에는 정기적금 우대이율 조건에 대한 정보가 포함되...          0.000000   
2  해당 내용은 담당 직원 확인 후 안내하겠습니다(②). 추가적인 정보가 필요하시면 언...          0.000000   
5  청년에게 유리한 예금 상품으로는 '청년희망적금'과 '청년 주택드림 청약통장'이 있습...          0.471018   
0  OTP 비밀번호 오류 해제 방법은 다음과 같습니다:\n\n1. 우리WON뱅킹에 접속...      

## Example

In [8]:
for i in range(5):
    print('\nuser_input:',df['user_input'][i])
    print('retrieved_contexts:',df['retrieved_contexts'][i])
    print('response:',df['response'][i])
    print('='*120)


user_input: OTP 비밀번호 오류 해제 방법 알려줘
retrieved_contexts: ['Q: 보안카드(OTP발생기) 비밀번호 오입력횟수가 초과되었어요. 인터넷뱅킹에서 오류해제 가능한가요?\nA: 보안카드 비밀번호 연속 3회 또는 OTP발생기 비밀번호 연속 10회 오입력 초과 시 보안매체를 이용할 수 없습니다. 우리WON뱅킹에서 비대면 실명확인 절차로 오류해제 가능합니다. (인터넷뱅킹에서는 불가)', 'Q: 이용자비밀번호 3회 오류 상태인데, 인터넷뱅킹에서 오류 해제 할 수 있나요?\nA: 기존 이용자비밀번호를 알고 있다면 인터넷뱅킹에서 오류해제 할 수 있습니다. (보안매체 및 인증서 입력 필요)', '▶ 인터넷뱅킹 경로 안내 : 개인뱅킹 로그인→ (아이디 로그인 영역) 비밀번호 재등록→ 이용자비밀번호 3회 오류해제', '※ 이용자비밀번호 3회오류인 경우, 아이디로그인은 불가하나 인증서 등 다른 방법의 로그인은 가능합니다.(법인 제외)', '※ 인터넷뱅킹에서 오류해제가 불가하다면, 우리WON뱅킹에서 비대면 실명확인 절차로 오류해제 및 비밀번호 재등록을 할 수 있습니다.', 'Q: 이용자비밀번호(로그인비밀번호) 2회 오류 상태인데, 인터넷뱅킹에서 재등록 할 수 있나요?\nA: 인터넷뱅킹에서 이용자비밀번호 재등록이 가능하며, 보안매체와 인증서 입력이 필요합니다.', '', '▶ 이용자비밀번호 재등록 경로 안내 : 개인인터넷뱅킹 → (아이디 로그인 영역) 비밀번호재등록 → 이용자비밀번호 재등록', '', '※ 인터넷뱅킹에서 재등록 불가하다면, 우리WON뱅킹에서 비대면 실명확인 절차로 오류해제 및 비밀번호 재등록을 할 수 있습니다.']
response: OTP 비밀번호 오류 해제 방법은 다음과 같습니다:

1. 우리WON뱅킹에 접속합니다.
2. 비대면 실명확인 절차를 진행합니다.
3. 오류가 해제된 후, 필요한 경우 비밀번호를 재등록합니다.

※ 인터넷뱅킹에서는 OTP 비밀번호 오류 해제가 불가능합니다(①).

user_input: 해외에서 인터넷뱅킹 이